Importing data from a csv file

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('Hackathon_data.csv')


Cleaning the data

In [3]:

new = df['Elevation Interval of Screen(masl)'].str.split(" - ", n=1, expand = True) #Split the screen interval into a top and bottom, in a new DataFrame

df['Screen Top(masl)'] = new[0] #Create new columns in the original data 
df['Screen Bot(masl)'] = new[1] 


df['Date'] = df['Date'].str[3:] #Change date from dd-mm-yy to mm-yy

df = df.set_index(['Well name'])

df = df[df['Corrected Groundwater Surface Elevation(masl)'].str[0] != "<"] #Delete all gw elevation data starting with "<"
      
df.head()


,Date,Ground Elevation(masl),Datum Elevation(masl),Surveyed Stickup(m),Total Depth of Piezometer(mbgs),Bottom of Piezometer Elevation(masl),Depth Interval of Screen(mbgs),Elevation Interval of Screen(masl),Depth Interval of Sand(mbgs),Elevation Interval of Sand(masl),...,Corrected Depth to Groundwater(mbtoc),Corrected Depth to Groundwater(mbgs),Observed Groundwater Surface Elevation(masl),Corrected Groundwater Surface Elevation(masl),Depth to Bottom of Well(mbtoc),Measured Stick up(m),Easting(m),Northing(m),Screen Top(masl),Screen Bot(masl)
Well name,,,,,,,,,,,,,,,,,,,,,
43629,Jun-16,953.5,954.39,0.88,7.8,945.7,4.80 - 7.80,948.70 - 945.70,NaN,NaN,...,8.2,7.32,946.19,946.19,8.69,N/M,645463.505,5867925.859,948.70,945.70
43629,Oct-16,953.5,954.39,0.88,7.8,945.7,4.80 - 7.80,948.70 - 945.70,NaN,NaN,...,6.39,5.51,948,948,N/M,N/M,645463.505,5867925.859,948.70,945.70
43629,May-17,953.5,954.39,0.88,7.8,945.7,4.80 - 7.80,948.70 - 945.70,NaN,NaN,...,6.42,5.54,947.97,947.97,N/M,N/M,645463.505,5867925.859,948.70,945.70
43629,Oct-17,953.5,954.39,0.88,7.8,945.7,4.80 - 7.80,948.70 - 945.70,NaN,NaN,...,6.94,6.06,947.45,947.45,8.7,N/M,645463.505,5867925.859,948.70,945.70
43629,May-18,953.5,954.39,0.88,7.8,945.7,4.80 - 7.80,948.70 - 945.70,NaN,NaN,...,7.62,6.74,946.77,946.77,8.7,N/M,645463.505,5867925.859,948.70,945.70


Narrowing down the data to a single date and relevant columns only

In [23]:

df = df[df['Date']=='Jun-16'] #For a select constant date, gather all well information

#Clean dataframe to selected columns
df = df[['Date', 'Corrected Groundwater Surface Elevation(masl)', 'Easting(m)','Northing(m)','Screen Top(masl)','Screen Bot(masl)']]

#Change type of Screen Top and Base to numeric
df['Screen Top(masl)'] = pd.to_numeric(df['Screen Top(masl)'])
df['Screen Bot(masl)'] = pd.to_numeric(df['Screen Bot(masl)'])

#Add screen midpoint
df['Screen Mid(masl)'] = (df['Screen Top(masl)'] + df['Screen Bot(masl)'])/2

df.head()


,Date,Corrected Groundwater Surface Elevation(masl),Easting(m),Northing(m),Screen Top(masl),Screen Bot(masl),Screen Mid(masl)
Well name,,,,,,,
43629,Jun-16,946.19,645463.505,5867925.859,948.70,945.70,947.200
TH7,Jun-16,946.85,645461.893,5867883.754,946.21,945.36,945.785
35217,Jun-16,948.82,645617.050,5867782.746,948.80,945.80,947.300
35401,Jun-16,947.27,645466.248,5867809.039,947.82,944.82,946.320
97-21,Jun-16,947.19,645608.630,5867740.071,947.14,943.14,945.140


In [27]:
#df.iloc[0,1]

'946.19'

Not Necessary: Write a new csv with clean data

In [129]:
#df.to_csv('clean_data_jun16.csv')

# Clustering

On to clustering..

In [24]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

In [29]:
# Select data to use for clustering
X = df.iloc[:,2:-1]
print(X.head())

### we need to scale the data
X = StandardScaler().fit_transform(X)

           Easting(m)  Northing(m)  Screen Top(masl)  Screen Bot(masl)
Well name                                                             
43629      645463.505  5867925.859            948.70            945.70
TH7        645461.893  5867883.754            946.21            945.36
35217      645617.050  5867782.746            948.80            945.80
35401      645466.248  5867809.039            947.82            944.82
97-21      645608.630  5867740.071            947.14            943.14


K-means clustering

In [30]:
### K-means clustering

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
print("Label:", kmeans.labels_)

df['Group'] = kmeans.labels_
print("Cluster centers", kmeans.cluster_centers_)

Label: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 2 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0
 0 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1
 1 2]
Cluster centers [[ 0.24667021  0.24928488  0.64597518  0.66135263]
 [-0.72300224 -1.26145594 -0.32098464 -0.39523214]
 [ 0.08876054  0.66664702 -1.47668569 -1.43943987]]


In [31]:
df.head()

,Date,Corrected Groundwater Surface Elevation(masl),Easting(m),Northing(m),Screen Top(masl),Screen Bot(masl),Screen Mid(masl),Group
Well name,,,,,,,,
43629,Jun-16,946.19,645463.505,5867925.859,948.70,945.70,947.200,0
TH7,Jun-16,946.85,645461.893,5867883.754,946.21,945.36,945.785,0
35217,Jun-16,948.82,645617.050,5867782.746,948.80,945.80,947.300,0
35401,Jun-16,947.27,645466.248,5867809.039,947.82,944.82,946.320,0
97-21,Jun-16,947.19,645608.630,5867740.071,947.14,943.14,945.140,0


# Visualization